**Описание проекта**

Вы работаете в интернет-магазине «Стримчик», который продаёт по всему миру
компьютерные игры. Из открытых источников доступны исторические данные о
продажах игр, оценки пользователей и экспертов, жанрыи платформы (например,
Xbox илиPlayStation). Вам нужно выявить определяющие успешность игры
закономерности. Это позволит сделать ставку на потенциально популярный продукт и
спланировать рекламные кампании.

Перед вами данные до 2016 года. Представим, что сейчас декабрь 2016 г., и вы
планируете кампанию на 2017-й. Нужно отработать принцип работы с данными.
Неважно, прогнозируете ли вы продажи на 2017 год поданным 2016-го или же 2027-й
— по данным 2026 года.

В наборе данных попадается аббревиатура ESRB (Entertainment Software Rating Board) — это ассоциация, определяющая возрастной рейтинг компьютерных игр. ESRB
оценивает игровой контент и присваивает ему подходящую возрастную категорию,
например, «Для взрослых», «Для детей младшего возраста» или «Для подростков».

# **1. Предобработка данных**

In [ ]:
import pandas as pd
import numpy as np
import math
import plotly.express as px
import statistics
from tabulate import tabulate
from scipy.stats import ttest_ind
import statsmodels

Загружаем датасет с разделителем данных в виде запятой.

In [ ]:
data = pd.read_csv('games.csv', sep=',')

Замением названия столбцов и приведим к нижнему регистру

In [ ]:
new_columns = list(data.columns.str.lower())

In [ ]:
data.columns = new_columns

Проверяем типы данных и количество пропусков:

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16715 non-null  object 
 2   year_of_release  16446 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16715 non-null  float64
 5   eu_sales         16715 non-null  float64
 6   jp_sales         16715 non-null  float64
 7   other_sales      16715 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       10014 non-null  object 
 10  rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


# **2. Обработка пропущенных значений и типов данных.**

**У нас представлены данные:**
1. name, platform, genre и rating типом object;
2. year_of_release и critic_score являются int64, но из-за пропусков записаны как float64;
3. остальные были правильно заданы как float64.
4. user_score записан как object из-за того, что в этой колонке присутствует значение 'tbd' (рейтинг игры еще не определен). Заменим его на оценку критиков или на NaN. Оценки критиков это целые числа от 0 до 100, то чтобы сравнивать оценки в одном диапазоне, разделим имеющиеся оценки критиков на 10.
5. user_score это оценка пользователей и она может быть от 0.0 до 10.0, поэтому заменим на float64. 
6. Заменим оценки критиков на значения деленные на 10. Чтоб оба типа оценок были в одном диапазоне.
7. Дополним имеющиеся данные столбцом суммарных продаж во всех регионах.

Заменим значение 'tbd' на оценку критиков деленную на десять или на NaN.

In [ ]:
data['user_score'] = [np.nan if math.isnan(data.loc[row, 'critic_score']) 
                      else data.loc[row, 'critic_score'] / 10
                      if data.loc[row, 'user_score'] == 'tbd' or math.isnan(data.loc[row, 'user_score'] 
                      if type(data.loc[row, 'user_score']) != str else 0)
                      else data.loc[row, 'user_score']
                      for row in range(len(data))]

Заменим формат данных на float64

In [ ]:
data['user_score'] = data['user_score'].astype('float64')

Заменим значения оценки критиков на диапазон от 0 до 10.

In [ ]:
data['critic_score'] = data['critic_score'] / 10

Дополним столбцом суммарных продаж во всех регионах: 

In [ ]:
data['sum_sales'] = data['eu_sales'] + data['jp_sales'] + \
                    data['na_sales'] + data['other_sales']

In [ ]:
data.head(1)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,sum_sales
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,7.6,8.0,E,82.54


# **3. Есть ли в данных выбросы?**

Для проверки этого факта построим описание всего дата сета.

In [ ]:
data.describe()

,year_of_release,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,sum_sales
count,16446.000000,16715.000000,16715.000000,16715.000000,16715.000000,8137.000000,8137.000000,16715.000000
mean,2006.484616,0.263377,0.145060,0.077617,0.047342,6.896768,7.032543,0.533396
std,5.877050,0.813604,0.503339,0.308853,0.186731,1.393816,1.452663,1.548261
min,1980.000000,0.000000,0.000000,0.000000,0.000000,1.300000,0.500000,0.000000
25%,2003.000000,0.000000,0.000000,0.000000,0.000000,6.000000,6.200000,0.060000
50%,2007.000000,0.080000,0.020000,0.000000,0.010000,7.100000,7.300000,0.170000
75%,2010.000000,0.240000,0.110000,0.040000,0.030000,7.900000,8.100000,0.470000
max,2016.000000,41.360000,28.960000,10.220000,10.570000,9.800000,9.600000,82.540000


На первый взгляд кажется, что в продажах есть выбросы, однако есть уникальные игры, которые были сверх популярны и имеют большое число продаж. 

Чтобы разобраться в этом построим для каждого региона бокс плот и выведем топ игр.

**Для Северной Америки:**

In [ ]:
fig = px.box(data, y="na_sales")
fig.show()

Выведем отдельно топ 5 игр по продажам, чтоб убедиться, что эти игры не случайно попали туда и не являются выбросами.

In [ ]:
list_top_na_sales = list(data.sort_values(by='na_sales', ascending=False).reset_index(drop=True).loc[:4, 'name'])
print(list_top_na_sales)

['Wii Sports', 'Super Mario Bros.', 'Duck Hunt', 'Tetris', 'Mario Kart Wii']


Все 5 игр популячны и не являются выбросами в данных.

**Для Европы:**

In [ ]:
fig = px.box(data, y="eu_sales")
fig.show()

Аналогично, для Европы видим, что топ 5 игр не являются выбросами.

In [ ]:
list_top_eu_sales = list(data.sort_values(by='eu_sales', ascending=False).reset_index(drop=True).loc[:4, 'name'])
print(list_top_eu_sales)

['Wii Sports', 'Mario Kart Wii', 'Nintendogs', 'Wii Sports Resort', 'Brain Age: Train Your Brain in Minutes a Day']


**Для Японии:**

In [ ]:
fig = px.box(data, y="jp_sales", labels={"jp_sales": "Japan Sales"})
fig.show()

В Японии популярны игры про покемонов и Марио, что тоже не является выбросами из данных.

In [ ]:
list_top_jp_sales = list(data.sort_values(by='jp_sales', ascending=False).reset_index(drop=True).loc[:4, 'name'])
print(list_top_jp_sales)

['Pokemon Red/Pokemon Blue', 'Pokemon Gold/Pokemon Silver', 'Super Mario Bros.', 'New Super Mario Bros.', 'Pokemon Diamond/Pokemon Pearl']


**Остальные продажи:**

In [ ]:
fig = px.box(data, y="other_sales")
fig.show()

В остальных продажах полученные игры также являются супер популярными и могут иметь столько продаж.

In [ ]:
list_top_other_sales = list(data.sort_values(by='other_sales', ascending=False
                                             ).reset_index(drop=True
                                                           ).loc[:4, 'name'])
print(list_top_other_sales)

['Grand Theft Auto: San Andreas', 'Wii Sports', 'Gran Turismo 4', 'Grand Theft Auto V', 'Mario Kart Wii']


# **4. Cколько игр выпускалось в разные годы. Важны ли данные за все периоды?**

Для поиска распределения количества игр по годам построим pivot_table и уже по полученным данным построим бар плот.

In [ ]:
table_game_year = data.pivot_table(index='year_of_release', values='genre',
                                   aggfunc='count', fill_value=0)
table_game_year.head(1)

,genre
year_of_release,
1980.0,9


In [ ]:
fig = px.bar(table_game_year, x=table_game_year.index, y='genre')
fig.show()

Помимо графика построим описательную статистику для этого распределения:

In [ ]:
table_game_year.describe()

,genre
count,37.000000
mean,444.432432
std,451.604334
min,9.000000
25%,36.000000
50%,338.000000
75%,762.000000
max,1427.000000


Из графика видно, что количество игр росло до 2000 года и с 2001 вышло на средний уровень (выше 444 игр в год). Посмотрим также распределение количества жанров по годам, чтобы увидеть когда сформировался основной список.

In [ ]:
table_genre_year = data.pivot_table(index='year_of_release', values='genre',
                                   aggfunc='nunique', fill_value=0)
table_genre_year.head(1)

,genre
year_of_release,
1980.0,5


Построим зависимость возникновения уникальных жанров от года, чтоб выяснить когда они были сформированы. 

In [ ]:
fig = px.bar(table_genre_year, x=table_genre_year.index, y='genre')
fig.show()

Исходя из этого графика видно, что начиная с 1991 года уже была сформирована классификация игр. 

Таким образом можно считать, что ограничение данных с 2001 года содержит в себе полные данные о жанрах игр. Однако также стоит посмотреть на динамику возникновения игр по разным жанрам, чтоб убедиться, что вся картина в целом видна для предсказания количества игр в 2017 году.

In [ ]:
data2001 = data[data['year_of_release'] >= 2001]

Рассмотрим распределение игр по жанрам с 2001 года.



In [ ]:
table_kind_genre = data2001.pivot_table(index=['year_of_release','genre'], 
                                         values='platform', aggfunc='count', 
                                         fill_value=0).reset_index()
table_kind_genre.head(1)

,year_of_release,genre,platform
0,2001.0,Action,67


In [ ]:
fig = px.bar(table_kind_genre, x='year_of_release', y='platform',color='genre')
fig.show()

Видно, что за последние 5 лет (с 2012 по 2016) количество игр устаканилось. И практически не меняется со временем.

In [ ]:
data2011 = data[data['year_of_release'] > 2011]

Уточним полученное распредление и добавим процентное распределение игр по каждому из жанров.

In [ ]:
table1 = data2011.pivot_table(index=['year_of_release','genre'], 
                                         values='platform', aggfunc='count', 
                                         fill_value=0).reset_index().sort_values(by='year_of_release')
table1.head(1)

,year_of_release,genre,platform
0,2012.0,Action,265


Для этого добавим колонку для общего числа игр для каждого года.

In [ ]:
years_list = list(table1.year_of_release.unique())
for year in years_list:
  small_temp_data = table1[table1['year_of_release'] == year]
  temp_index = list(small_temp_data.index)
  sum_games = small_temp_data.platform.sum()
  table1.loc[temp_index, 'sum_year'] = sum_games
table1.head(1)

,year_of_release,genre,platform,sum_year
0,2012.0,Action,265,653.0


После этого уже посчитаем процентное соотношение для каждого жанра.

In [ ]:
table1['percent'] = table1.platform / table1.sum_year * 100 

In [ ]:
table1.head(1)

,year_of_release,genre,platform,sum_year,percent
0,2012.0,Action,265,653.0,40.58193


Построим бар плот для 2002-2016 годов.

In [ ]:
fig = px.bar(table1, x='year_of_release', y='percent',color='genre')
fig.show()

**Вывод:** 
1. Для прогноза количества игр на 2017 год нужны данные об играх за последние 5 лет.
2. Для построения жизни платформ нужны данные с 2001 года.

# **5. За какой характерный срок появляются новые и исчезают старые платформы?**

Для этого построим среднее время жизни платформ с 2001 года.

In [ ]:
table3 = data2001.pivot_table(values='year_of_release', index='platform', 
                 aggfunc=lambda x: x.max()-x.min()+1)
table3.sort_values(by='year_of_release', ascending=False)

,year_of_release
platform,
PC,16.0
PSP,12.0
X360,12.0
PS3,11.0
Wii,11.0
PS2,11.0
DS,10.0
XB,8.0
DC,8.0


In [ ]:
table3.describe()

,year_of_release
count,20.00000
mean,7.25000
std,4.14062
min,1.00000
25%,4.00000
50%,7.00000
75%,11.00000
max,16.00000


Таким образом можно сказать, что новые платформы живут в среднем от 7 до 11 лет.

# **6. Распределение продаж по платформам.**

Посмотрим на распределение суммарных продаж по платформам и выделим из них список с наибольшими суммарными продажами.

In [ ]:
table2 = data2011.pivot_table(index=[ 'platform'], 
                                         values='sum_sales', aggfunc='sum', 
                                         fill_value=0).reset_index()
table2.sort_values(by='sum_sales', ascending=False)

,platform,sum_sales
4,PS4,314.14
3,PS3,288.79
9,X360,236.54
0,3DS,194.61
10,XOne,159.32
8,WiiU,82.19
2,PC,62.65
6,PSV,49.18
7,Wii,35.37
1,DS,12.55


Видно, что первые 5 платформ имеют в несколько раз большие продажи чем остальные. Посмотрим целиком всю динамику для первых 10 платформ.

In [ ]:
table_platf_year = table2.sort_values(by='sum_sales', ascending=False).iloc[:10, 
                                                                            :]

Построим распределение продаж по годам для интересующих нас платформ.

In [ ]:
table_platf_year_sum = data2001.pivot_table(index=['year_of_release', 'platform'], 
                                         values='sum_sales', aggfunc='sum', 
                                         fill_value=0).reset_index()
table_platf_year_sum.sort_values(by='sum_sales', ascending=False).head(15)

,year_of_release,platform,sum_sales
28,2004.0,PS2,211.81
74,2009.0,Wii,206.97
16,2002.0,PS2,205.38
22,2003.0,PS2,184.31
66,2008.0,Wii,171.32
82,2010.0,X360,170.03
7,2001.0,PS2,166.43
35,2005.0,PS2,160.66
87,2011.0,PS3,156.78
57,2007.0,Wii,152.77


In [ ]:
list_platforms = list(table_platf_year.platform.unique())
temp_data = table_platf_year_sum[table_platf_year_sum['platform'].isin(list_platforms)]

In [ ]:
fig = px.line(temp_data, x='year_of_release', y='sum_sales', color='platform')
fig.show()

Таким образом, мы имеем топ 5 платформ по продажам, и считаем их потенциально прибыльными. Список ниже. На 2016 год все продажи падают, что видно из рисунка выше. 


In [ ]:
table2.sort_values(by='sum_sales', ascending=False).reset_index().loc[:5,'platform']

0     PS4
1     PS3
2    X360
3     3DS
4    XOne
5    WiiU
Name: platform, dtype: object

# **7. Распределение глобальных продаж игр в разбивке по платформам.**

●Постройте график «ящик с усами» по глобальным продажамигр в разбивке по
платформам. Опишите результат.
●Посмотрите, как влияют на продажи внутри одной популярнойплатформы
отзывы пользователей и критиков. Постройте диаграммурассеяния и
посчитайте корреляцию между отзывами и продажами.Сформулируйте
выводы.
●Соотнесите выводы с продажами игр на других платформах.
●Посмотрите на общее распределение игр по жанрам. Чтоможно сказать о
самых прибыльных жанрах? Выделяются ли жанры с высокимии низкими
продажами?

In [ ]:
temp_data = data2001[data2001['platform'].isin(list_platforms)]

In [ ]:
fig = px.box(temp_data, x="platform", y="sum_sales")
fig.show()

Как видно из графика boxplot что в целом все ящики имеют схожие границы, и платформы отличаются только выбросами. Выведем уточняющую статистику:

In [ ]:
list_mean_sales = [data2001[data2001['platform'] == platf].sum_sales.mean()
for platf in list_platforms]
list_median_sales = [data2001[data2001['platform'] == platf].sum_sales.median()
for platf in list_platforms]
list_75_quant_sales = [data2001[data2001['platform'] == platf].sum_sales.quantile(0.75)
for platf in list_platforms]

In [ ]:
print('Среднее значение для суммарных продаж по платформам', 
      statistics.mean(list_mean_sales))
print('Средняя медиана для суммарных продаж по платформам', 
      statistics.mean(list_median_sales))
print('Среднее значение 75% всех суммарных продаж по платформам меньше чем', 
      statistics.mean(list_75_quant_sales))

Среднее значение для суммарных продаж по платформам 0.5420422582895368
Средняя медиана для суммарных продаж по платформам 0.171
Среднее значение 75% всех суммарных продаж по платформам меньше чем 0.484


Таким образом видно, что различия в продажах для разных платформ лишь в эксклюзивах у той или иной платформы. Большинство продаж располагается в одинаковых пределах.

# **8. Влияние отзывов пользователей и критиков на продажи внутри одной популярной платформы.**

Как влияют на продажи внутри одной популярной платформы
отзывы пользователей и критиков? Построим диаграмму рассеяния и
посчитаем корреляцию между отзывами и продажами. Будем рассматривать две платформы из топ 5, а именно для PS4 и X360.

# **8.1 Посмотрим влияние отзывов на суммарные продажи для PS4.**

In [ ]:
data2001.head(1)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,sum_sales
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,7.6,8.0,E,82.54


In [ ]:
data_clear2001 = data2001[(data2001['critic_score'].notna())&(data2001['platform'] == 'PS4')]

In [ ]:
fig = px.scatter(data_clear2001, x='critic_score', y='sum_sales', 
                 trendline="ols")

fig.show()

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



Корреляция между оценками критиков и суммарными продажами слабая, R = 0.4. Оценка взята с графика. Следовательно R^2=0.16, или другими словами critic_score определяет sum_sales на 16%.

In [ ]:
data_clear_user2001 = data2001[(data2001['user_score'].notna())&(data2001['platform'] == 'PS4')]

In [ ]:
fig = px.scatter(data_clear_user2001, x='user_score', y='sum_sales', 
                 trendline="ols")

fig.show()

Корреляция между оценками юзеров и суммарными продажами очень слабая, R = 0.033. Оценка взята с графика. Следовательно R^2=0.0011, или другими словами user_score определяет sum_sales на 0.11%.


# **8.2 Посмотрим влияние отзывов на суммарные продажи для X360.**

In [ ]:
data_clear2001 = data2001[(data2001['critic_score'].notna())&(data2001['platform'] == 'X360')]

In [ ]:
fig = px.scatter(data_clear2001, x='critic_score', y='sum_sales', 
                 trendline="ols")

fig.show()

Корреляция между оценками критиков и суммарными продажами слабая, R = 0.4. Оценка взята с графика. Следовательно R^2=0.15, или другими словами critic_score определяет sum_sales на 15%.

In [ ]:
data_clear_user2001 = data2001[(data2001['user_score'].notna())&(data2001['platform'] == 'X360')]

In [ ]:
fig = px.scatter(data_clear_user2001, x='user_score', y='sum_sales', 
                 trendline="ols")

fig.show()

Корреляция между оценками юзеров и суммарными продажами слабая, R = 0.12. Оценка взята с графика. Следовательно R^2=0.014, или другими словами user_score определяет sum_sales на 1.4%.


# **8.3 Посмотрим влияние оценок на продажи на PC.**

In [ ]:
data_clear2001 = data2001[(data2001['critic_score'].notna())&(data2001['platform'] == 'PC')]

In [ ]:
fig = px.scatter(data_clear2001, x='critic_score', y='sum_sales', 
                 trendline="ols")

fig.show()

Корреляция между оценками критиков и суммарными продажами слабая, R = 0.24. Оценка взята с графика. Следовательно R^2=0.06, или другими словами critic_score определяет sum_sales на 6%.

In [ ]:
data_clear_user2001 = data2001[(data2001['user_score'].notna())&(data2001['platform'] == 'PC')]

In [ ]:
fig = px.scatter(data_clear_user2001, x='user_score', y='sum_sales', 
                 trendline="ols")

fig.show()

Корреляция между оценками юзеров и суммарными продажами слабая, R = 0.009. Оценка взята с графика. Следовательно R^2=0.000075, или другими словами user_score определяет sum_sales на 0.0075%.

**Выводы**
1. Для платформ PS4 и X360 существует слабая связь между оценками критиков и суммарными продажами. R = 15-16%
2. Для PC critic_score определяет sum_sales на 6%. Что более чем в два раза хуже чем для PS4 и X360.
3. Для всех трех платформ не прослеживается связь между оценками юзеров и суммарными продажами.



# **9. Распределение игр по жанрам и продажам, связь жанров с прибылью.**

Построим общее распределение игр по жанрам с целью выделить жанры с высокими и низкими продажами. Рассмотрим это распределение за последние 5 лет, так как в разделе 4 было показано, что именно в последние 5 лет распределение жанров игр не сильно варьируется. Лучшей визуализацией таких результатов будет круговая диграмма. Так как суммарные продажи игр имеют высокие выбросы, что должно исказить картину, строить графики будем для средних продаж и для общих сумм.

In [ ]:
table_genre_mean_sales = data2011.pivot_table(index=['year_of_release', 'genre'], 
                     values='sum_sales', 
                     aggfunc='mean').reset_index()
table_genre_mean_sales.head(1)

,year_of_release,genre,sum_sales
0,2012.0,Action,0.45


In [ ]:
fig = px.pie(table_genre_mean_sales, values='sum_sales', names='genre')
fig.show()

Для средних продаж выделяется 3 жанра с наибольшим процентом продаж, выше 10% от общего вклада. Это shooter, platform и sports.

В это же время игры имеющие меньше 5% от всех продаж это strategy, puzzle и adventure. Эти жанры можно отнести к самым низким продажам.

In [ ]:
table_genre_sales_2011 = data2011.pivot_table(index=['genre'], 
                     values='sum_sales', 
                     aggfunc='sum').reset_index()
table_genre_sales_2011.head(1)

,genre,sum_sales
0,Action,441.12


In [ ]:
fig = px.pie(table_genre_sales_2011, values='sum_sales', names='genre')
fig.show()

Для суммарных продаж выделяется 4 жанра составляющие большую часть всех продаж. Это action, shooter, role-playing и sports. 

К самым малопродающимся играм можно отнести strategy и puzzle.

**Выводы**

1. В топ игр по средним продажам можно отнести такие жанры как: shooter, platform и sports.
2. Жанры action и role-playing тоже вошли в топ из-за того, что в этих жанрах были уникальные игры выдвинувшие их в топ по суммарным продажам.
3. Самые малоприбыльные жанры это strategy, puzzle и adventure. Они находятся внизу топа как по суммарным продажам так и по средним значениям.


# **10.  Портрет пользователя каждого региона (NA, EU, JP)**

Для составления портрета мы воспользуемся такими параметрами как:

1. Самые популярные платформы (топ-5) и доли продаж.
2. Самые популярные жанры (топ-5). 
3. Влияение рейтинга ESRB на продажи. 
M — Mature; E — Everyone; E10+ — Everyone 10+; T — Teen. 

Из полученных данных для удобства создадим таблицу.

**Портрет пользователя для Северной Америки**

In [ ]:
def generate_table_sales(col_name, region):
  sales = region + '_sales'
  temp_table = data2011.pivot_table(index=[col_name], 
                                         values=sales, aggfunc='sum', 
                                         fill_value=0)
  result_table = temp_table.sort_values(by=sales, 
                              ascending=False).reset_index()
  if len(result_table) >= 5:
    result_table = result_table.loc[:4,]
  return sales, list(result_table[col_name].unique())

In [ ]:
def generate_table(data_table, dict_table, region):
  temp_list = ['platform', 'genre', 'rating']
  for col_name in temp_list:
    name_col = col_name + ' for ' + region
    sales_name, dict_table[name_col] = generate_table_sales(col_name, region)
    if col_name == 'rating':
      fig = px.box(data_table, x=col_name, y=sales_name)
      fig.show()
  return dict_table

In [ ]:
dict_table = {}

In [ ]:
for region in ['na', 'eu', 'jp']:
  dict_table = generate_table(data2011, dict_table, region) 

Видно, что выбросы и средние значения весьма четко связаны с рейтингом. В таблицу мы запишем распределение рейтинга в зависимости от суммарных продаж.

**Таким образом полученная статистика по трем регионам представляется в таком виде:**

In [ ]:
print(tabulate(dict_table, headers='keys', tablefmt='fancy_grid'))

╒═══════════════════╤════════════════╤═════════════════╤═══════════════════╤════════════════╤═════════════════╤═══════════════════╤════════════════╤═════════════════╕
│ platform for na   │ genre for na   │ rating for na   │ platform for eu   │ genre for eu   │ rating for eu   │ platform for jp   │ genre for jp   │ rating for jp   │
╞═══════════════════╪════════════════╪═════════════════╪═══════════════════╪════════════════╪═════════════════╪═══════════════════╪════════════════╪═════════════════╡
│ X360              │ Action         │ M               │ PS4               │ Action         │ M               │ 3DS               │ Role-Playing   │ E               │
├───────────────────┼────────────────┼─────────────────┼───────────────────┼────────────────┼─────────────────┼───────────────────┼────────────────┼─────────────────┤
│ PS4               │ Shooter        │ E               │ PS3               │ Shooter        │ E               │ PS3               │ Action         │ T               

**Выводы.**
1. Общие статистики для Северной Америки и Европы схожи. Первые 4 жанра из топа одинаковые. Распределение количества продаж по рейтингу тоже. 
2. Список платформ в топе для Северной Америки и Европы одинаков, но располагается в разном порядке.
3. Самые главные отличия состоят в Японской аудитории. Там в списке платформ добавляется платформа WiiU и PSV. Ранее их не было. Также пропадают платформы X360 и Xone. Жанры и распределения рейтинга также отличаются от остальных регионов. Для Японии появляются такие жанры как Misc, Simulation и Fighting. 

# **11. Средние пользовательские рейтинги платформ Xbox One и PC одинаковые** 

Проверим гипотезу, что средние пользовательские рейтинги платформ Xbox One и PC одинаковые. Для этого воспользуемся t-тестом. Так как само указание на платформу ограничивает данные по годам, то нет смысла вырезать данные с 2001 или 2011 года.

In [ ]:
# Example of the Student's t-test
data_xbox = data[data['platform'] == 'XB'].user_score
data_pc = data[data['platform'] == 'PC'].user_score
stat, p = ttest_ind(data_xbox, data_pc)
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('Probably the same distribution')
else:
	print('Probably different distributions')

stat=8.356, p=0.000
Probably different distributions


p-value очень мало, а значит утверждение, что средние пользовательские рейтинги для этих платформ различны. Посмотрим насколько они отличаются.

In [ ]:
data_xbox.mean() - data_pc.mean()

1.228168547277762

# **12. Средние пользовательские рейтинги жанров Action и Sports разные.**

Проверим гипотезу, что средние пользовательские рейтинги жанров Action и Sports разные. Для этого воспользуемся t-тестом. Стоит отдельно отметить, что мы можем проверить это утверждение на всех данных с 2001 года. Или же только за последние 5 лет. Ключевая разница в том, что последние годы в среднем мы имеем стабильное число игр в году. А с 2001 года уже появились все жанры и можно считать данные полными для построения гипотез.

In [ ]:
# Example of the Student's t-test
data_action = data2001[data2001['genre'] == 'Action'].user_score.notna()
data_sports = data2001[data2001['genre'] == 'Sports'].user_score.notna()
stat, p = ttest_ind(data_action, data_sports)
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('Probably the same distribution')
else:
	print('Probably different distributions')

stat=3.921, p=0.000
Probably different distributions


Таким образом для данных с 2001 года можно утверждать, что средние отличны.

In [ ]:
# Example of the Student's t-test
data_action_2011 = data2011[data2011['genre'] == 'Action'].user_score.notna()
data_sports_2011 = data2011[data2011['genre'] == 'Sports'].user_score.notna()
stat, p = ttest_ind(data_action_2011, data_sports_2011)
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('Probably the same distribution')
else:
	print('Probably different distributions')

stat=-3.259, p=0.001
Probably different distributions


Аналогично данным с 2001, с 2011 года можно утверждать, что средние отличны.
Вероятность того, что средние отличны всего на 0.1 процент случайны.


# **13. Общий вывод**

**Выводы:**
1. Был обработан датасет состоящий из 16715 строк и 11 колонок.
2. Было выяснено, что основные жанры игр были сформированы уже в 1991 году. С 2001 года среднее количество игр вышло на уровень выше 450+ игр в год и это можно считать полными данными.
С 2011 года данные пригодны для прогноза количества игр на 2017 год.
3. Новые платформы живут в среднем от 7 до 11 лет. Это включает в себя этап пика и спада.
4. Топ 5 платформ по продажам с 2001 по 2016 годы PS4, PS3, X360, 3DS, XOne, WiiU. Платформы указаны в порядке убывания.
5. Для топ 5 платформ общие продажи лежат в схожих границах. Главные отличия это в выбросах, в самых популярных играх.
6. Для платформ PS4 и X360 существует слабая связь между оценками критиков и суммарными продажами, корреляция R = 15-16.Для PC critic_score определяет sum_sales на 6%. Что более чем в два раза хуже чем для PS4 и X360. Для всех трех платформ не прослеживается связь между оценками юзеров и суммарными продажами.
7. В топ игр по средним продажам можно отнести такие жанры как: shooter, platform и sports. Жанры action и role-playing тоже вошли в топ из-за того, что в этих жанрах были уникальные игры выдвинувшие их в топ по суммарным продажам.
8. Самые малоприбыльные жанры это strategy, puzzle и adventure. Они находятся внизу топа как по суммарным продажам так и по средним значениям.
9. Распределение по рейтингу игр для продаж для Северной Америки и Европы: M — Mature; E — Everyone; E10+ — Everyone 10+; T — Teen.. Однако для Японии это распределение другое:  E — Everyone; T — Teen; M — Mature; E10+ — Everyone 10+. Более подробная статистика по играм в разделе 10.
10. Средние пользовательские рейтинги платформ Xbox One и PC  p-value очень мало, а значит утверждение, что средние пользовательские рейтинги для этих платформ различны. Посмотрим насколько они отличаются.
11. Средние пользовательские рейтинги жанров Action и Sports разные. Аналогично данным с 2001, с 2011 года можно утверждать, что средние отличны.
Вероятность того, что средние отличны всего на 0.1 процент случайны.